# Why is Autograd Important?

Before we understand what Autograd is, let's first understand why we need it.

## A Simple Example

Let's start with a basic mathematical function:

$$
y = x^2
$$

Here, $y$ is a function of $x$. Now, suppose you're given the task of writing a Python program that calculates the derivative of $y$ with respect to $x$ (i.e., $\frac{dy}{dx}$) for any given value of $x$.

This is simple. The derivative of $x^2$ is $2x$. So:
- If $x = 2$, then $\frac{dy}{dx} = 4$
- If $x = 3$, then $\frac{dy}{dx} = 6$

You can easily write a Python function to do that.

## More Complex Functions

But what happens when the function becomes more complex? For example:

$$
y = x^2 \\
z = \sin(y)
$$

Now, if you need to calculate $\frac{dz}{dx}$, it becomes a bit harder. You now need to apply the chain rule of calculus:

$$
\frac{dz}{dx} = \frac{dz}{dy} \cdot \frac{dy}{dx}
$$

This requires you to first calculate derivatives of inner functions, then multiply them. As the number of nested functions increases, the derivative expressions become more and more complex.

## Even More Complex Chains

Now imagine an even more complicated chain:

$$
a = f(b), \quad b = g(c), \quad c = h(x)
$$

To find $\frac{da}{dx}$, you now need to calculate:

$$
\frac{da}{dx} = \frac{da}{db} \cdot \frac{db}{dc} \cdot \frac{dc}{dx}
$$

Manually calculating and coding such derivatives is difficult and error-prone, especially as the number of layers increases.

## Connection to Deep Learning

This becomes especially relevant in deep learning. Let's say we're training a neural network to predict whether a student gets placed based on their CGPA. Even the simplest neural network involves multiple steps:

### Forward Pass:
1. Compute $z = wx + b$
2. Apply activation: $\hat{y} = \sigma(z)$
3. Compute loss $L(\hat{y}, y)$

Here, the output (loss) is a nested function of the inputs and weights.

### Backward Pass (Backpropagation):
To update weights using gradient descent, you need:
- $\frac{\partial L}{\partial w}$
- $\frac{\partial L}{\partial b}$

To compute these, you again apply the chain rule across multiple layers and functions.

If the network becomes large (as in real-world deep learning), manually computing all these derivatives becomes practically impossible.

## So, Why Autograd?

Autograd (short for "Automatic Differentiation") is a core feature of PyTorch that automatically calculates derivatives for you.

- It records operations you perform on tensors
- Then it automatically applies the chain rule to compute gradients
- You don't have to manually derive or code any gradients
- It makes training neural networks efficient, scalable, and less error-prone

### Formal Definition:
Autograd is a core component of PyTorch that provides automatic differentiation for all operations on tensors. It dynamically builds a computational graph and computes gradients using the chain rule during the backward pass.

## Summary:
- Deep learning models are essentially nested functions
- Training them requires calculating derivatives using chain rule
- As models become deeper, doing this manually is nearly impossible
- Autograd automates this process and makes deep learning practical

In [1]:
def dy_dx(x):
  return 2*x

In [2]:
dy_dx(3)

6

In [3]:
import torch

In [4]:
x = torch.tensor(3.0, requires_grad=True)

In [5]:
y = x**2

In [6]:
x

tensor(3., requires_grad=True)

In [7]:
y

tensor(9., grad_fn=<PowBackward0>)

In [8]:
y.backward()

In [9]:
x.grad

tensor(6.)

In [10]:
import math

def dz_dx(x):
    return 2 * x * math.cos(x**2)

In [11]:
dz_dx(4)

-7.661275842587077

In [12]:
x = torch.tensor(4.0, requires_grad=True)

In [13]:
y = x ** 2

In [14]:
z = torch.sin(y)

In [15]:
x

tensor(4., requires_grad=True)

In [16]:
y

tensor(16., grad_fn=<PowBackward0>)

In [17]:
z

tensor(-0.2879, grad_fn=<SinBackward0>)

In [18]:
z.backward()

In [19]:
x.grad

tensor(-7.6613)

In [20]:
y.grad

<ipython-input-20-10b3a7061f6d>:1: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at /pytorch/build/aten/src/ATen/core/TensorBody.h:489.)
  y.grad


In [21]:
import torch

# Inputs
x = torch.tensor(6.7)  # Input feature
y = torch.tensor(0.0)  # True label (binary)

w = torch.tensor(1.0)  # Weight
b = torch.tensor(0.0)  # Bias

In [22]:
# Binary Cross-Entropy Loss for scalar
def binary_cross_entropy_loss(prediction, target):
    epsilon = 1e-8  # To prevent log(0)
    prediction = torch.clamp(prediction, epsilon, 1 - epsilon)
    return -(target * torch.log(prediction) + (1 - target) * torch.log(1 - prediction))

In [23]:
# Forward pass
z = w * x + b  # Weighted sum (linear part)
y_pred = torch.sigmoid(z)  # Predicted probability

# Compute binary cross-entropy loss
loss = binary_cross_entropy_loss(y_pred, y)

In [24]:
loss

tensor(6.7012)

In [25]:
# Derivatives:
# 1. dL/d(y_pred): Loss with respect to the prediction (y_pred)
dloss_dy_pred = (y_pred - y)/(y_pred*(1-y_pred))

# 2. dy_pred/dz: Prediction (y_pred) with respect to z (sigmoid derivative)
dy_pred_dz = y_pred * (1 - y_pred)

# 3. dz/dw and dz/db: z with respect to w and b
dz_dw = x  # dz/dw = x
dz_db = 1  # dz/db = 1 (bias contributes directly to z)

dL_dw = dloss_dy_pred * dy_pred_dz * dz_dw
dL_db = dloss_dy_pred * dy_pred_dz * dz_db

In [26]:
print(f"Manual Gradient of loss w.r.t weight (dw): {dL_dw}")
print(f"Manual Gradient of loss w.r.t bias (db): {dL_db}")

Manual Gradient of loss w.r.t weight (dw): 6.691762447357178
Manual Gradient of loss w.r.t bias (db): 0.998770534992218


In [27]:
x = torch.tensor(6.7)
y = torch.tensor(0.0)

In [28]:
w = torch.tensor(1.0, requires_grad=True)
b = torch.tensor(0.0, requires_grad=True)

In [29]:
w

tensor(1., requires_grad=True)

In [ ]:
b

tensor(0., requires_grad=True)

In [ ]:
z = w*x + b
z

tensor(6.7000, grad_fn=<AddBackward0>)

In [ ]:
y_pred = torch.sigmoid(z)
y_pred

tensor(0.9988, grad_fn=<SigmoidBackward0>)

In [ ]:
loss = binary_cross_entropy_loss(y_pred, y)
loss

tensor(6.7012, grad_fn=<NegBackward0>)

In [ ]:
loss.backward()

In [ ]:
print(w.grad)
print(b.grad)

tensor(6.6918)
tensor(0.9988)


In [ ]:
x = torch.tensor([1.0, 2.0, 3.0], requires_grad=True)

In [ ]:
x

tensor([1., 2., 3.], requires_grad=True)

In [ ]:
y = (x**2).mean()
y

tensor(4.6667, grad_fn=<MeanBackward0>)

In [ ]:
y.backward()

In [ ]:
x.grad

tensor([0.6667, 1.3333, 2.0000])

In [ ]:
# clearing grad
x = torch.tensor(2.0, requires_grad=True)
x

tensor(2., requires_grad=True)

In [ ]:
y = x ** 2
y

tensor(4., grad_fn=<PowBackward0>)

In [ ]:
y.backward()

In [ ]:
x.grad

tensor(4.)

In [ ]:
x.grad.zero_()

tensor(0.)

In [ ]:
# disable gradient tracking
x = torch.tensor(2.0, requires_grad=True)
x

tensor(2., requires_grad=True)

In [ ]:
y = x ** 2
y

tensor(4., grad_fn=<PowBackward0>)

In [ ]:
y.backward()

In [ ]:
x.grad

tensor(4.)

In [ ]:
# option 1 - requires_grad_(False)
# option 2 - detach()
# option 3 - torch.no_grad()

In [ ]:
x.requires_grad_(False)

tensor(2.)

In [ ]:
x

tensor(2.)

In [ ]:
y = x ** 2

In [ ]:
y

tensor(4.)

In [ ]:
y.backward()

RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

In [ ]:
x = torch.tensor(2.0, requires_grad=True)
x

tensor(2., requires_grad=True)

In [ ]:
z = x.detach()
z

tensor(2.)

In [ ]:
y = x ** 2

In [ ]:
y

tensor(4., grad_fn=<PowBackward0>)

In [ ]:
y1 = z ** 2
y1

tensor(4.)

In [ ]:
y.backward()

In [ ]:
y1.backward()

RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

In [ ]:
x = torch.tensor(2.0, requires_grad=True)
x

tensor(2., requires_grad=True)

In [ ]:
y = x ** 2

In [ ]:
y

tensor(4., grad_fn=<PowBackward0>)

In [ ]:
y.backward()